In [2]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import shutil
import numpy as np
import pandas as pd
import os
import random

In [15]:
poke_data = '/content/drive/MyDrive/PokemonData/'
classes = os.listdir(poke_data)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os

def load_images_from_folder(poke_data):
    images = []
    for filename in os.listdir(poke_data):
        img = cv2.imread(os.path.join(poke_data,filename))
        if img is not None:
            images.append(img)
    return images

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
generator = ImageDataGenerator(rotation_range=20,
                               rescale = 1./255,
                               width_shift_range=0.1,zoom_range=0.1,
                               horizontal_flip=True,height_shift_range=0.1,
                               shear_range=0.1,
                               fill_mode='nearest', validation_split=0.2)

In [22]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
image_shape = (256,256,3)
model = Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(BatchNormalization())
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(512,activation='relu'))
model.add(Dense(150,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       1

In [26]:
batch_size = 16

In [29]:
train= generator.flow_from_directory(poke_data,subset='training',
                                                target_size=image_shape[:2],
                                                batch_size=batch_size,
                                                class_mode='categorical'
                                                color_mode='rgb',
                                                )

Found 5511 images belonging to 150 classes.


In [31]:
test = generator.flow_from_directory(poke_data,subset='validation',
                                                target_size=image_shape[:2],
                                                class_mode='categorical', color_mode='rgb',
                                                batch_size=batch_size,)

Found 1309 images belonging to 150 classes.


In [32]:
early_stopping = EarlyStopping(monitor='val_loss',patience=3)

In [40]:
history= model.fit(train_image_gen,epochs=10,validation_data=test_image_gen,callbacks=[early_stopping])

Epoch 1/10
345/345 [==============================] - 133s 385ms/step - loss: 1.2046 - accuracy: 0.6505 - val_loss: 1.8628 - val_accuracy: 0.5332
Epoch 2/10
345/345 [==============================] - 132s 384ms/step - loss: 1.2377 - accuracy: 0.6405 - val_loss: 1.8343 - val_accuracy: 0.5393
Epoch 3/10
345/345 [==============================] - 133s 384ms/step - loss: 1.1818 - accuracy: 0.6645 - val_loss: 1.8141 - val_accuracy: 0.5500
Epoch 4/10
345/345 [==============================] - 133s 384ms/step - loss: 1.1659 - accuracy: 0.6630 - val_loss: 1.8375 - val_accuracy: 0.5531
Epoch 5/10
345/345 [==============================] - 133s 385ms/step - loss: 1.1238 - accuracy: 0.6703 - val_loss: 1.8631 - val_accuracy: 0.5348
Epoch 6/10
345/345 [==============================] - 133s 386ms/step - loss: 1.1229 - accuracy: 0.6692 - val_loss: 1.8748 - val_accuracy: 0.5432


In [ ]:
print(model.evaluate(test_image_gen))

82/82 [==============================] - 24s 292ms/step - loss: 1.9430 - accuracy: 0.5294
[1.9430240392684937, 0.529411792755127]


In [ ]:
loss= pd.DataFrame(model.history.history)

In [ ]:
loss.to_csv('loss-summary.csv')

In [ ]:
model.save('/content/drive/MyDrive/poke_model.h5')

In [ ]:
model = load_model("/content/drive/MyDrive/poke_model.h5")

In [ ]:
# plotting the loss
loss[['loss','val_loss']].plot()

In [ ]:
# plotting the accuracy
loss[['accuracy','val_accuracy']].plot()

In [ ]:
# Importing standard libraries for image and web url
from tensorflow.keras.preprocessing import image
from PIL import Image
import urllib.request

In [ ]:
def predict(img_url):
  url = img_url
  test_image_path = urllib.request.urlopen(url)
  img = Image.open(test_image_path)
  img = img.resize((200,200))
  plt.imshow(img)
  plt.show()
  img = np.array(img)
  if img.shape[2] != 3:
      img = img[:,:,:3]
  img = img/255
  my_img_array = np.expand_dims(img,axis=0)
  pred_class = np.argmax(model.predict(my_img_array),axis=-1)
  poke_class = {value:key for key,value in train_image_gen.class_indices.items()}
  print(poke_class[pred_class[0]])

In [ ]:
img = 'http://www.pngmart.com/files/12/Pokemon-Charizard-PNG-Image.png'
img2 = 'https://www.pngkey.com/png/full/149-1492440_pokemon-jigglypuff-png-jigglypuff-pokemon.png'
img3 = 'https://smallimg.pngkey.com/png/small/62-623293_pokemon-conquest-pokemon-art.png'
img4 = 'https://smallimg.pngkey.com/png/small/1-16272_pokemon-logo-png-transparent-pokemon-vector.png'
img5 = 'https://www.pngkey.com/png/full/62-622689_fire-pokemon-png-moltres-pokemon.png'
img6 = 'https://www.pngkey.com/png/full/66-668441_jpg-library-library-redo-ultimate-mewtwo-by-pokemon.png'

In [ ]:
pokemon = [img,img2,img3,img4,img5,img6]

for i in pokemon:
  pokemon(i)